<a href="https://colab.research.google.com/github/valentinamgavirira/reto-4/blob/main/Reto4analitica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install vaderSentiment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk #natural language toolkit
from nltk.tokenize import word_tokenize, sent_tokenize #separar frases y palabras
nltk.download('punkt_tab')#optimizador del proceso de tokenizacion
import spacy
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
# Cargar modelo de NLP
nlp = spacy.load('en_core_web_sm')
# Inicializar el analizador de sentimientos VADER
sia = SentimentIntensityAnalyzer()

# Escala de sentimientos
vn = ["Amenaza", "Pesimismo", "Inestabilidad", "Escepticismo", "Indiferente", "Neutral", "Favorable", "Optimismo", "Convicción", "Consolidación", "Confianza"]
vnm = np.array([-1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1.0])

# Función para analizar sentimientos
def analyze_sentiments_vader(text, year):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    data = []
    for sentence in sentences:
        print(sentence)
        score = sia.polarity_scores(sentence)
        print(score)
        compound = score['compound']
        print("la polaridad es:",score['compound'])
        d = np.abs(vnm - compound)
        categoria = np.where(d == min(d))[0][0]
        print(vn[categoria])
        sentiment = vn[categoria]
        data.append([sentence, compound, sentiment, year])
    return data
# Diccionario con los archivos de los informes
files = {
    "2020": "/content/InformeANALDEX_2020.txt",
    "2021": "/content/InformeANALDEX_2021.txt",
    "2022": "/content/InformeANALDEX_2022.txt",
    "2023": "/content/InformeANALDEX_2023.txt",
}
# Procesar los informes
results = []
for year, file_path in files.items():
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
        results.extend(analyze_sentiments_vader(text, year))

# Crear DataFrame
df = pd.DataFrame(results, columns=["Frase", "Polaridad", "Sentimiento", "Año"])

# Guardar en Excel
output_file = "Analisis_Sentimientos.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df.to_excel(writer, sheet_name="Sentimientos", index=False)

    # Crear gráficos
    plt.figure(figsize=(10, 5))
    sns.histplot(df, x="Polaridad", hue="Año", multiple="dodge", bins=20)
    plt.title("Histograma de Frecuencias de Polaridad")
    plt.savefig("histograma_polaridad.png")
    plt.close()

    sentiment_counts = df.groupby(["Año", "Sentimiento"]).size().unstack()
    sentiment_counts.plot(kind="bar", stacked=False, figsize=(10, 5))
    plt.title("Evolución de Sentimientos por Año")
    plt.savefig("evolucion_sentimientos.png")
    plt.close()

print(f"Análisis completado. Archivo guardado como {output_file}")

Informe Detallado del Comercio Exterior de Colombia en 2020
1.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
la polaridad es: 0.0
Neutral
Introducción
El año 2020 representó un punto de inflexión para la economía global, y Colombia no fue la excepción.
{'neg': 0.109, 'neu': 0.891, 'pos': 0.0, 'compound': -0.296}
la polaridad es: -0.296
Indiferente
La irrupción de la pandemia de COVID-19 generó una crisis sanitaria sin precedentes que impactó drásticamente las cadenas de suministro, las relaciones comerciales internacionales y la demanda interna.
{'neg': 0.228, 'neu': 0.772, 'pos': 0.0, 'compound': -0.8271}
la polaridad es: -0.8271
Pesimismo
En este contexto, el comercio exterior colombiano presentó una serie de particularidades que vale la pena analizar en profundidad.

2.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
la polaridad es: 0.0
Neutral
El Impacto de la Pandemia COVID-19 en el Comercio Exterior Colombiano
La pandemia de COVID-19 tuvo un efecto devastador en la

In [4]:
# Contar cuántas frases hay en cada sentimiento
sentimiento_counts = df['Sentimiento'].value_counts()

# Mostrar el resultado
print("Cantidad de frases por sentimiento:")
print(sentimiento_counts)

# Si quieres guardar este conteo en el archivo Excel
with pd.ExcelWriter(output_file, mode='a') as writer:
    sentimiento_counts.to_excel(writer, sheet_name="Conteo Sentimientos")

Cantidad de frases por sentimiento:
Sentimiento
Neutral          149
Inestabilidad      8
Indiferente        4
Optimismo          2
Pesimismo          1
Escepticismo       1
Name: count, dtype: int64


In [5]:
# Crear DataFrame
df = pd.DataFrame(results, columns=["Frase", "Polaridad", "Sentimiento", "Año"])

# Crear DataFrame de resumen
summary_df = df.groupby(["Año", "Sentimiento"]).size().unstack(fill_value=0).reset_index()
print("Resumen de sentimientos por año:")
print(summary_df)

# Guardar en Excel
output_file = "Analisis_Sentimientos2.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df.to_excel(writer, sheet_name="Sentimientos", index=False)
    summary_df.to_excel(writer, sheet_name="Resumen", index=False)

    # Crear gráficos
    plt.figure(figsize=(10, 5))
    sns.histplot(df, x="Polaridad", hue="Año", multiple="dodge", bins=20)
    plt.title("Histograma de Frecuencias de Polaridad")
    plt.savefig("histograma_polaridad.png")
    plt.close()

    sentiment_counts = df.groupby(["Año", "Sentimiento"]).size().unstack()
    sentiment_counts.plot(kind="bar", stacked=False, figsize=(10, 5))
    plt.title("Evolución de Sentimientos por Año")
    plt.savefig("evolucion_sentimientos.png")
    plt.close()

print(f"Análisis completado. Archivo guardado como {output_file}")

Resumen de sentimientos por año:
Sentimiento   Año  Escepticismo  Indiferente  Inestabilidad  Neutral  \
0            2020             1            1              3       37   
1            2021             0            1              1       44   
2            2022             0            1              2       33   
3            2023             0            1              2       35   

Sentimiento  Optimismo  Pesimismo  
0                    2          1  
1                    0          0  
2                    0          0  
3                    0          0  
Análisis completado. Archivo guardado como Analisis_Sentimientos2.xlsx


In [6]:
# Función para analizar sentimientos
def analyze_sentiments_vader(text, year):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    data = []
    for sentence in sentences:
        score = sia.polarity_scores(sentence)
        compound = score['compound']
        categoria = np.abs(vnm - compound).argmin()
        sentiment = vn[categoria]
        data.append([sentence, compound, sentiment, year])
    return data, len(word_tokenize(text)), len(sentences)

# Procesar los informes
results = []
summary_data = []
for year, file_path in files.items():
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
        data, word_count, sentence_count = analyze_sentiments_vader(text, year)
        results.extend(data)
        positive_words = sum(1 for d in data if d[1] > 0)
        negative_words = sum(1 for d in data if d[1] < 0)
        summary_data.append([year, word_count, sentence_count, positive_words, negative_words])

# Crear DataFrame de análisis detallado
df = pd.DataFrame(results, columns=["Frase", "Polaridad", "Sentimiento", "Año"])

# Crear DataFrame de resumen
summary_df = pd.DataFrame(summary_data, columns=["Año", "Palabras Analizadas", "Frases Analizadas", "Palabras Positivas", "Palabras Negativas"])
print("Resumen de palabras y frases analizadas por año:")
print(summary_df)
print(df)
# Guardar en Excel
output_file = "Analisis_Sentimientos1.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df.to_excel(writer, sheet_name="Sentimientos", index=False)
    summary_df.to_excel(writer, sheet_name="Resumen", index=False)

Resumen de palabras y frases analizadas por año:
    Año  Palabras Analizadas  Frases Analizadas  Palabras Positivas  \
0  2020                  856                 45                   2   
1  2021                  755                 46                   0   
2  2022                  742                 36                   0   
3  2023                  752                 38                   0   

   Palabras Negativas  
0                   6  
1                   2  
2                   3  
3                   3  
                                                 Frase  Polaridad  \
0    Informe Detallado del Comercio Exterior de Col...     0.0000   
1    Introducción\nEl año 2020 representó un punto ...    -0.2960   
2    La irrupción de la pandemia de COVID-19 generó...    -0.8271   
3    En este contexto, el comercio exterior colombi...     0.0000   
4    El Impacto de la Pandemia COVID-19 en el Comer...    -0.5574   
..                                                 ...       

De acuerdo con el analisis de los reportes anuales publicados por la Asociación Nacional de Exportadores (ANALDEX) entre los años 2020-2024,se destaca que la mayoria de las frases tuvieron un sentimiento neutral con 149 frases, seguido de inestabilidad con tan solo 8 frases. Esto en general de todos los años. En cuanto a cada año el que presento mas tipos de sentimientos fue el 2020 con una mayor cantidad de frases neutrales despues una mayor cantidad en inestabilidad, seguido por optimismo, pesimismo, esceptisimo e indiferencia; mientras que en el año 2021, 2022, 2023 solo tuvimos sentimientos neutrales, inestables e indiferentes,la mayor cantidad neutrales seguido de la inestabilidad con un poco mas de cantidad que indiferente   